# Stable Diffusion 3.5 Medium - Grass Species Generation

Generating scientific documentation images for multiple grass species (Brachiaria, Panicum, Cynodon) with different health conditions using positive/negative prompts.

## Species and Conditions:
- **Brachiaria brizantha**: 5 health conditions (saudável → severamente deteriorada)
- **Panicum maximum**: 5 health conditions (saudável → severamente deteriorado) 
- **Cynodon dactylon**: 5 health conditions (saudável → severamente deteriorado)

**Total**: 15 different conditions across 3 species, 2 images each = 30 images

In [ ]:
# Install required packages
!pip install diffusers transformers accelerate torch torchvision huggingface_hub --quiet

In [ ]:
# Import libraries
import torch
from diffusers import StableDiffusion3Pipeline
from huggingface_hub import login
import matplotlib.pyplot as plt
from PIL import Image
import os
from datetime import datetime

In [ ]:
# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Hugging Face login (required for SD 3.5 Medium)
login()

In [ ]:
# Load the Stable Diffusion 3.5 Medium model
print("Loading Stable Diffusion 3.5 Medium...")
pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    torch_dtype=torch.float16,
    device_map="balanced"
)

print("Model loaded successfully!")

In [ ]:
# Define prompts for all species and health conditions with improved naturalness

# Brachiaria prompts with health conditions
brachiaria_prompts = {
    'brachiaria_healthy_condition': {
        'positive': (
            "overhead view of healthy brachiaria brizantha pasture, "
            "thick bright green grass with vigorous growth, "
            "dense uniform coverage, no bare soil visible, "
            "wide leaf blades, robust plant structure, "
            "natural tufted growth pattern, optimal density, "
            "scientific documentation, agricultural research, "
            "natural daylight, top-down perspective, organic texture"
        ),
        'negative': (
            "ground level, thin grass, sparse coverage, "
            "yellow patches, bare soil, weeds, invasive plants, "
            "artistic photography, decorative lawn, synthetic, artificial, "
            "overly smooth, plastic appearance, computer generated look, "
            "unrealistic uniformity, digital artifacts"
        ),
        'description': "Brachiaria condição 1 - saudável"
    },
    'brachiaria_slight_decline': {
        'positive': (
            "aerial view of brachiaria pasture with minor stress signs, "
            "mostly green grass with some lighter colored patches, "
            "good overall coverage but slightly irregular density, "
            "tropical grass showing early stress indicators, "
            "scientific field documentation, research photography, "
            "natural lighting, overhead perspective, authentic field conditions"
        ),
        'negative': (
            "ground perspective, severe damage, completely yellow, "
            "ornamental setting, artistic effects, synthetic, "
            "overly processed, artificial lighting, plastic look"
        ),
        'description': "Brachiaria condição 2 - leve declínio"
    },
    'brachiaria_moderate_degradation': {
        'positive': (
            "top-down view of moderately degraded brachiaria pasture, "
            "mixed green and yellowish grass patches, "
            "uneven plant density, some small bare soil areas, "
            "tropical grass showing clear stress signs, "
            "reduced vigor compared to healthy pasture, "
            "agricultural study documentation, field research, natural variation"
        ),
        'negative': (
            "ground level, completely healthy, lush green, "
            "decorative grass, artistic photography, artificial, "
            "synthetic appearance, digital rendering"
        ),
        'description': "Brachiaria condição 3 - degradação moderada"
    },
    'brachiaria_advanced_degradation': {
        'positive': (
            "overhead documentation of severely degraded brachiaria, "
            "sparse grass coverage with extensive bare soil patches, "
            "yellowing and browning vegetation, weak plant structure, "
            "significant soil compaction visible, erosion signs, "
            "tropical pasture in advanced deterioration state, "
            "scientific documentation of pasture decline, realistic field conditions"
        ),
        'negative': (
            "healthy vegetation, dense coverage, bright green, "
            "artistic style, decorative setting, artificial, "
            "synthetic, computer generated, unrealistic"
        ),
        'description': "Brachiaria condição 4 - degradação avançada"
    },
    'brachiaria_severely_deteriorated': {
        'positive': (
            "aerial view of severely deteriorated brachiaria pasture, "
            "mostly bare compacted soil with scattered dying grass, "
            "brown and yellow stressed vegetation, soil erosion, "
            "failed tropical grassland, extreme degradation, "
            "invasive weeds present, poor soil structure visible, "
            "agricultural research documentation of pasture failure, natural deterioration"
        ),
        'negative': (
            "healthy pasture, green vegetation, good coverage, "
            "ornamental lawn, artistic photography, synthetic, "
            "artificial appearance, digital rendering"
        ),
        'description': "Brachiaria condição 5 - severamente deteriorada"
    }
}

# Panicum maximum prompts with health conditions
panicum_prompts = {
    'panicum_healthy_condition': {
        'positive': (
            "overhead view of healthy panicum maximum pasture, "
            "tall robust tropical grass with wide leaf blades, "
            "dense green coverage, vigorous upright growth, "
            "thick stems visible, large leaf structure, "
            "natural clumping pattern, optimal plant density, "
            "scientific documentation, agricultural research, "
            "natural daylight, top-down perspective, organic texture"
        ),
        'negative': (
            "ground level, short grass, thin blades, sparse coverage, "
            "yellowing, bare soil, weeds, temperate species, "
            "artificial, synthetic, computer generated, plastic appearance"
        ),
        'description': "Panicum condição 1 - saudável"
    },
    'panicum_slight_decline': {
        'positive': (
            "aerial view of panicum pasture with early stress signs, "
            "mostly green tall grass with some pale patches, "
            "good coverage but irregular growth vigor, "
            "large tropical grass showing initial decline, "
            "scientific field documentation, research photography, "
            "natural field conditions, authentic texture"
        ),
        'negative': (
            "ground perspective, severe damage, short grass, "
            "ornamental lawn, artistic effects, artificial, "
            "synthetic appearance, digital artifacts"
        ),
        'description': "Panicum condição 2 - leve declínio"
    },
    'panicum_moderate_degradation': {
        'positive': (
            "top-down view of moderately degraded panicum pasture, "
            "mixed green and yellow tall grass areas, "
            "uneven plant height, some bare patches visible, "
            "tropical grass with reduced vigor and density, "
            "signs of overgrazing stress, field research documentation, "
            "natural variation, authentic degradation patterns"
        ),
        'negative': (
            "healthy dense coverage, uniform green, "
            "decorative setting, artistic photography, "
            "synthetic, artificial, computer generated"
        ),
        'description': "Panicum condição 3 - degradação moderada"
    },
    'panicum_advanced_degradation': {
        'positive': (
            "overhead view of severely degraded panicum pasture, "
            "sparse tall grass with extensive bare soil, "
            "yellowing stems, weak plant structure, "
            "soil compaction and erosion visible, "
            "advanced deterioration of tropical grassland, "
            "realistic field conditions, natural decay patterns"
        ),
        'negative': (
            "lush vegetation, dense coverage, healthy growth, "
            "ornamental grass, artistic style, synthetic, "
            "artificial appearance, digital rendering"
        ),
        'description': "Panicum condição 4 - degradação avançada"
    },
    'panicum_severely_deteriorated': {
        'positive': (
            "aerial documentation of failed panicum pasture, "
            "mostly bare compacted soil, scattered dying tall grass, "
            "brown withered stems, extreme soil degradation, "
            "invasive weeds colonizing failed grassland, "
            "complete pasture system collapse, natural deterioration"
        ),
        'negative': (
            "healthy pasture, green vegetation, good structure, "
            "decorative lawn, artistic photography, synthetic, "
            "artificial, computer generated appearance"
        ),
        'description': "Panicum condição 5 - severamente deteriorado"
    }
}

# Cynodon dactylon prompts with health conditions
cynodon_prompts = {
    'cynodon_healthy_condition': {
        'positive': (
            "overhead view of healthy cynodon dactylon pasture, "
            "dense carpet-like grass coverage, uniform green color, "
            "fine textured stoloniferous growth, tight mat formation, "
            "no visible soil, excellent ground coverage, "
            "scientific documentation, agricultural research, "
            "natural lighting, top-down perspective, organic texture"
        ),
        'negative': (
            "ground level, coarse grass, sparse coverage, "
            "bare patches, tall growth, weeds, synthetic, "
            "artificial appearance, computer generated, plastic look"
        ),
        'description': "Cynodon condição 1 - saudável"
    },
    'cynodon_slight_decline': {
        'positive': (
            "aerial view of cynodon pasture with minor stress, "
            "mostly dense green carpet with some thin areas, "
            "good overall coverage but slight color variation, "
            "stoloniferous grass showing early stress indicators, "
            "scientific field documentation, research photography, "
            "natural field conditions, authentic texture"
        ),
        'negative': (
            "ground perspective, severe thinning, bare soil, "
            "ornamental turf, artistic effects, synthetic, "
            "artificial, digital artifacts"
        ),
        'description': "Cynodon condição 2 - leve declínio"
    },
    'cynodon_moderate_degradation': {
        'positive': (
            "top-down view of moderately degraded cynodon pasture, "
            "patchy grass coverage with visible thin areas, "
            "mixed green and pale sections, reduced density, "
            "stoloniferous grass with declining vigor, "
            "some bare soil patches becoming visible, "
            "natural degradation patterns, realistic field conditions"
        ),
        'negative': (
            "thick carpet coverage, uniform green, "
            "decorative lawn, artistic photography, "
            "synthetic, artificial, computer generated"
        ),
        'description': "Cynodon condição 3 - degradação moderada"
    },
    'cynodon_advanced_degradation': {
        'positive': (
            "overhead documentation of severely degraded cynodon, "
            "sparse patchy grass with significant bare areas, "
            "weakened stoloniferous growth, soil compaction, "
            "yellowing and browning grass patches, "
            "advanced deterioration of ground coverage, "
            "authentic degradation, natural field conditions"
        ),
        'negative': (
            "dense mat formation, healthy green color, "
            "ornamental setting, artistic style, synthetic, "
            "artificial appearance, digital rendering"
        ),
        'description': "Cynodon condição 4 - degradação avançada"
    },
    'cynodon_severely_deteriorated': {
        'positive': (
            "aerial view of failed cynodon pasture system, "
            "predominantly bare compacted soil, scattered grass remnants, "
            "brown dying stolons, severe erosion patterns, "
            "weed invasion in bare areas, complete system failure, "
            "agricultural documentation of pasture collapse, "
            "natural deterioration, realistic field conditions"
        ),
        'negative': (
            "carpet-like coverage, healthy stolons, green grass, "
            "decorative turf, artistic photography, synthetic, "
            "artificial, computer generated appearance"
        ),
        'description': "Cynodon condição 5 - severamente deteriorado"
    }
}

# Combine all prompts
all_prompts = {**brachiaria_prompts, **panicum_prompts, **cynodon_prompts}

print(f"Loaded {len(brachiaria_prompts)} Brachiaria prompts")
print(f"Loaded {len(panicum_prompts)} Panicum prompts")
print(f"Loaded {len(cynodon_prompts)} Cynodon prompts")
print(f"Total prompts: {len(all_prompts)}")
print("✓ Prompts updated with enhanced natural language for more realistic results")

In [ ]:
# Create output directory with timestamp and date organization
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
date_folder = datetime.now().strftime("%Y-%m-%d")
output_dir = f"grass_generation_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory: {output_dir}")
print(f"Date folder for organization: {date_folder}")

In [ ]:
# Function to generate images with improved parameters for more natural results
import random

def generate_species_images(prompt_data, num_images=2):
    """
    Generate images using positive and negative prompts with varied parameters for natural results
    """
    images = []
    
    positive_prompt = prompt_data['positive']
    negative_prompt = prompt_data['negative']
    
    print(f"Generating {num_images} images for: {prompt_data['description']}")
    print(f"Positive: {positive_prompt[:100]}...")
    print(f"Negative: {negative_prompt[:100]}...")
    
    for i in range(num_images):
        print(f"  Generating image {i+1}/{num_images}...")
        
        # Vary parameters for more natural, less robotic results
        guidance_scale = random.uniform(4.5, 6.5)  # Lower guidance for more natural results
        num_steps = random.randint(28, 35)         # More steps for better quality
        seed = random.randint(0, 2**32 - 1)       # Random seed for variety
        
        # Add slight variation to the prompt for more diversity
        prompt_variations = [
            positive_prompt,
            positive_prompt + ", natural variation, organic pattern",
            positive_prompt + ", realistic texture, field conditions",
            positive_prompt + ", authentic agricultural setting"
        ]
        
        selected_prompt = random.choice(prompt_variations)
        
        print(f"    Parameters: guidance={guidance_scale:.1f}, steps={num_steps}, seed={seed}")
        
        with torch.inference_mode():
            # Set the generator with our random seed
            generator = torch.Generator(device="cuda" if torch.cuda.is_available() else "cpu")
            generator.manual_seed(seed)
            
            image = pipe(
                prompt=selected_prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=num_steps,
                guidance_scale=guidance_scale,
                height=1024,
                width=1024,
                generator=generator
            ).images[0]
        
        images.append(image)
    
    return images

# Function to determine species from prompt key
def get_species_from_key(prompt_key):
    if prompt_key.startswith('brachiaria'):
        return 'brachiaria'
    elif prompt_key.startswith('panicum'):
        return 'panicum'
    elif prompt_key.startswith('cynodon'):
        return 'cynodon'
    else:
        return 'unknown'

# Advanced function with even more control for specific use cases
def generate_species_images_advanced(prompt_data, num_images=2, base_seed=None):
    """
    Advanced generation with more control over randomness and parameters
    """
    images = []
    
    positive_prompt = prompt_data['positive']
    negative_prompt = prompt_data['negative']
    
    print(f"Generating {num_images} images (Advanced) for: {prompt_data['description']}")
    
    for i in range(num_images):
        print(f"  Generating image {i+1}/{num_images}...")
        
        # If base_seed provided, use variations around it
        if base_seed is not None:
            seed = base_seed + i * 1000  # Systematic variation
        else:
            seed = random.randint(0, 2**32 - 1)  # Completely random
        
        # Different parameter sets for different "styles" of natural variation
        param_sets = [
            {"guidance": 4.0, "steps": 30, "style": "very_natural"},
            {"guidance": 5.0, "steps": 32, "style": "balanced"},
            {"guidance": 5.5, "steps": 28, "style": "controlled"},
            {"guidance": 4.5, "steps": 35, "style": "detailed"}
        ]
        
        param_set = random.choice(param_sets)
        
        print(f"    Style: {param_set['style']}, guidance={param_set['guidance']}, steps={param_set['steps']}, seed={seed}")
        
        with torch.inference_mode():
            generator = torch.Generator(device="cuda" if torch.cuda.is_available() else "cpu")
            generator.manual_seed(seed)
            
            image = pipe(
                prompt=positive_prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=param_set['steps'],
                guidance_scale=param_set['guidance'],
                height=1024,
                width=1024,
                generator=generator
            ).images[0]
        
        images.append(image)
    
    return images

In [ ]:
# Test improved generation with a sample prompt to check naturalness
print("Testing improved generation parameters...")

# Select a test prompt
test_prompt_key = 'brachiaria_healthy_condition'
test_prompt_data = all_prompts[test_prompt_key]

# Generate a test image with new parameters
print("Generating test images with improved natural parameters...")
test_images = generate_species_images(test_prompt_data, num_images=1)

# Display the test result
if test_images:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(8, 8))
    plt.imshow(test_images[0])
    plt.title(f"Test Result: {test_prompt_data['description']}")
    plt.axis('off')
    plt.show()
    print("✓ Test generation completed - Check if the result looks more natural!")
else:
    print("✗ Test generation failed")

In [ ]:
# Generate all images for all species
all_generated_images = {}
total_prompts = len(all_prompts)
current_prompt = 0

for prompt_key, prompt_data in all_prompts.items():
    current_prompt += 1
    species = get_species_from_key(prompt_key)
    print(f"\n{'='*50}")
    print(f"Processing {current_prompt}/{total_prompts}: {prompt_key} ({species})")
    print(f"{'='*50}")
    
    try:
        images = generate_species_images(prompt_data, num_images=2) # Generate 2 images per prompt set
        all_generated_images[prompt_key] = {
            'images': images,
            'description': prompt_data['description'],
            'species': species
        }
        print(f"✓ Successfully generated 2 images for {prompt_key}")
        
    except Exception as e:
        print(f"✗ Error generating {prompt_key}: {e}")
        all_generated_images[prompt_key] = {
            'images': [],
            'description': prompt_data['description'],
            'species': species,
            'error': str(e)
        }

print(f"\n\nGeneration complete! Generated images for {len([k for k, v in all_generated_images.items() if v['images']])} prompt sets.")

# Count by species
species_count = {}
for data in all_generated_images.values():
    species = data['species']
    if species not in species_count:
        species_count[species] = {'total': 0, 'successful': 0}
    species_count[species]['total'] += 1
    if data['images']:
        species_count[species]['successful'] += 1

for species, counts in species_count.items():
    print(f"{species.capitalize()}: {counts['successful']}/{counts['total']} prompt sets successful")

In [ ]:
# Display images in grid layout
def display_image_grid(all_images, cols=2):
    """
    Display all generated images in a grid layout
    """
    prompt_keys = [k for k, v in all_images.items() if v['images']]
    
    if not prompt_keys:
        print("No images to display")
        return
    
    rows = len(prompt_keys)
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, prompt_key in enumerate(prompt_keys):
        images = all_images[prompt_key]['images']
        description = all_images[prompt_key]['description']
        
        for j in range(min(cols, len(images))):
            axes[i, j].imshow(images[j])
            axes[i, j].set_title(f"{description} - Imagem {j+1}", fontsize=10)
            axes[i, j].axis('off')
        
        # Hide unused subplots
        for j in range(len(images), cols):
            axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/brachiaria_grid_overview.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Grid overview saved as: {output_dir}/brachiaria_grid_overview.png")

# Display the grid
display_image_grid(all_generated_images)

In [ ]:
# Save individual images organized by species
def save_individual_images_by_species(all_images, output_directory):
    """
    Save each generated image individually, organized by species
    """
    saved_count = 0
    species_folders = {}
    
    # Create species directories
    for prompt_key, data in all_images.items():
        species = data['species']
        if species not in species_folders:
            species_dir = os.path.join(output_directory, species)
            os.makedirs(species_dir, exist_ok=True)
            species_folders[species] = species_dir
            print(f"Created directory: {species_dir}")
    
    # Save images in respective species folders
    for prompt_key, data in all_images.items():
        if not data['images']:
            continue
        
        species = data['species']
        species_dir = species_folders[species]
        
        for i, image in enumerate(data['images']):
            filename = f"{prompt_key}_image_{i+1}.png"
            filepath = os.path.join(species_dir, filename)
            
            image.save(filepath)
            saved_count += 1
            print(f"Saved: {species}/{filename}")
    
    print(f"\nTotal images saved: {saved_count}")
    print(f"Images organized in {len(species_folders)} species folders")
    return saved_count, species_folders

# Save all images organized by species
saved_images, species_directories = save_individual_images_by_species(all_generated_images, output_dir)

In [ ]:
# Generate summary report by species
print("\n" + "="*60)
print("GRASS SPECIES GENERATION SUMMARY")
print("="*60)

successful_prompts = 0
total_images = 0

# Summary by species
for species in ['brachiaria', 'panicum', 'cynodon']:
    species_prompts = [k for k, v in all_generated_images.items() if v['species'] == species]
    species_successful = [k for k in species_prompts if all_generated_images[k]['images']]
    species_total_images = sum(len(all_generated_images[k]['images']) for k in species_prompts)
    
    print(f"\n{species.upper()}:")
    for prompt_key in species_prompts:
        data = all_generated_images[prompt_key]
        if data['images']:
            print(f"  ✓ {data['description']}: {len(data['images'])} images")
            successful_prompts += 1
            total_images += len(data['images'])
        else:
            print(f"  ✗ {data['description']}: Failed - {data.get('error', 'Unknown error')}")

print(f"\nOVERALL RESULTS:")
print(f"- Successful prompt sets: {successful_prompts}/{len(all_prompts)}")
print(f"- Total images generated: {total_images}")
print(f"- Species processed: {len(species_directories)}")
print(f"- Output directory: {output_dir}")
print(f"- Species folders: {list(species_directories.keys())}")
print("="*60)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
import os

# Function to create organized Google Drive folder structure
def create_google_drive_structure(local_output_dir, species_directories, date_folder):
    """
    Create organized folder structure in Google Drive:
    img_gen/
    ├── brachiaria/
    │   └── 2024-08-28/
    │       └── images
    ├── panicum/
    │   └── 2024-08-28/
    │       └── images
    └── cynodon/
        └── 2024-08-28/
            └── images
    """
    
    # Base Google Drive directory
    base_drive_dir = '/content/drive/MyDrive/img_gen'
    os.makedirs(base_drive_dir, exist_ok=True)
    
    print(f"Creating organized structure in Google Drive: {base_drive_dir}")
    print(f"Date folder: {date_folder}")
    
    copied_files = 0
    
    for species, local_species_dir in species_directories.items():
        # Create species folder in Google Drive
        drive_species_dir = os.path.join(base_drive_dir, species)
        os.makedirs(drive_species_dir, exist_ok=True)
        
        # Create date folder within species
        drive_date_dir = os.path.join(drive_species_dir, date_folder)
        os.makedirs(drive_date_dir, exist_ok=True)
        
        print(f"Created: {drive_date_dir}")
        
        # Copy all images from local species directory to Google Drive
        if os.path.exists(local_species_dir):
            for filename in os.listdir(local_species_dir):
                if filename.endswith('.png'):
                    local_file = os.path.join(local_species_dir, filename)
                    drive_file = os.path.join(drive_date_dir, filename)
                    shutil.copy2(local_file, drive_file)
                    copied_files += 1
                    print(f"  Copied: {species}/{date_folder}/{filename}")
    
    # Also create a summary grid overview in the base directory
    grid_overview_local = os.path.join(local_output_dir, "grass_grid_overview.png")
    if os.path.exists(grid_overview_local):
        grid_overview_drive = os.path.join(base_drive_dir, f"grass_overview_{date_folder}.png")
        shutil.copy2(grid_overview_local, grid_overview_drive)
        print(f"Copied overview: grass_overview_{date_folder}.png")
    
    return base_drive_dir, copied_files

# Create organized Google Drive structure
try:
    drive_base_dir, total_copied = create_google_drive_structure(
        output_dir, species_directories, date_folder
    )
    
    print(f"\n{'='*50}")
    print(f"GOOGLE DRIVE UPLOAD SUMMARY")
    print(f"{'='*50}")
    print(f"✓ Successfully copied {total_copied} images to Google Drive")
    print(f"✓ Base directory: {drive_base_dir}")
    print(f"✓ Organization: Species > Date > Images")
    print(f"✓ Date folder: {date_folder}")
    
    # Display folder structure
    print(f"\nFolder structure created:")
    for species in species_directories.keys():
        print(f"  img_gen/{species}/{date_folder}/")
        species_images = len([f for f in os.listdir(species_directories[species]) if f.endswith('.png')])
        print(f"    └── {species_images} images")
    
    print(f"{'='*50}")
    
except Exception as e:
    print(f"Error creating Google Drive structure: {e}")
    print(f"Attempting simple backup copy...")
    
    # Fallback: simple copy
    try:
        simple_drive_dir = f'/content/drive/MyDrive/img_gen_backup_{timestamp}'
        shutil.copytree(output_dir, simple_drive_dir, dirs_exist_ok=True)
        print(f"Backup copy successful: {simple_drive_dir}")
    except Exception as backup_error:
        print(f"Backup copy also failed: {backup_error}")